In [17]:
import os
import requests
from bs4 import BeautifulSoup
import re
import logging

# Логгирование
LOG_FILE = "parser.log"
logging.basicConfig(
    level=logging.INFO,  
    format="%(asctime)s [%(levelname)s] - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
        #logging.StreamHandler() - если нужно в console логи выводить
    ]
)

class WikiVoyageParser:
    def __init__(self, language="ru", save_path="data"):
        self.api_url = f"https://{language}.wikivoyage.org/w/api.php"
        self.save_path = save_path
        os.makedirs(save_path, exist_ok=True)

    def get_page_sections(self, title):
        """Получает список разделов (секций) статьи."""
        params = {
            "action": "parse",
            "format": "json",
            "page": title,
            "prop": "sections"
        }
        response = requests.get(self.api_url, params=params)
        data = response.json()

        sections = data.get("parse", {}).get("sections", [])
        if sections:
            logging.info(f"📑 Найдены разделы для {title}!")
        else:
            logging.warning(f"⚠️ Разделы для {title} не найдены!")

        return sections

    def get_section_content(self, title, section_id, section_name):
        """Получает текст конкретного раздела статьи и очищает его от HTML."""
        params = {
            "action": "parse",
            "format": "json",
            "page": title,
            "prop": "text",
            "section": section_id
        }
        response = requests.get(self.api_url, params=params)
        data = response.json()
        section_html = data.get("parse", {}).get("text", {}).get("*", "")

        clean_text = self.clean_html(section_html)
        clean_text = self.remove_section_redundancy(clean_text, section_name)

        return clean_text

    def clean_html(self, html_text):
        """Удаляет HTML-теги и исправляет переносы строк."""
        soup = BeautifulSoup(html_text, "html.parser")

        for tag in soup.find_all(["table", "sup", "style", "script"]):
            tag.decompose()

        blocks = []

        # Обрабатываем <p> и <li> отдельно
        for el in soup.find_all(["p", "li", "h3", "h2"]):
            txt = el.get_text(" ", strip=True)
            if txt:
                blocks.append(txt)
    
        # Склеиваем в текст с двойными переносами между блоками
        text = "\n\n".join(blocks)
    
        # Убираем служебные метки вида [править]
        text = re.sub(r'\[\s*[^]]*\]', '', text)

        return text.strip()

    def remove_section_redundancy(self, text, section_name):
        """Удаляет повторное упоминание заголовка раздела в начале текста."""
        pattern = rf'^{section_name}\s*'
        clean_text = re.sub(pattern, '', text, count=1, flags=re.IGNORECASE).strip()
        return clean_text

    def save_to_txt(self, country, place, sections):
        """Сохраняет два раздела в один файл."""
        place_path = os.path.join(self.save_path, country)
        os.makedirs(place_path, exist_ok=True)

        file_path = os.path.join(place_path, f"{place}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(f"{place.upper()}\n\n")  # Заголовок файла

            for sec in sections:
                section_title = sec["line"]
                section_id = sec["index"]

                if section_title in ["Достопримечательности", "Чем заняться"]:
                    content = self.get_section_content(place, section_id, section_title)

                    if content.strip():
                        f.write(f"### {section_title}\n\n")
                        f.write(content + "\n\n")

        logging.info(f"✅ Сохранено: {file_path}")

    def process_place(self, country, place):
        """Парсит информацию о месте и сохраняет нужные разделы."""
        logging.info(f"🔍 Обрабатываю: {place} ({country})")
        sections = self.get_page_sections(place)

        if not sections:
            logging.error(f"❌ Не удалось получить разделы для {place}")
            return
        
        self.save_to_txt(country, place, sections)
        logging.info(f"✅ {place} ({country}) сохранен!")

    def process_all(self, locations):
        """Проходит по странам и их городам, обрабатывая данные."""
        for country, cities in locations.items():
            logging.info(f"\n🌍 Обрабатываем страну: {country}")

            # Обрабатываем страну
            self.process_place(country, country)

            # Обрабатываем города
            for city in cities:
                self.process_place(country, city)

        logging.info("🎉 Все страны и города обработаны и сохранены!")

# Запускаем парсер
parser = WikiVoyageParser(language="ru")

# Список стран и городов
locations_to_parse = {
    "Россия": ["Москва", "Санкт-Петербург", "Казань"],
    "Франция": ["Париж", "Лион"],
    "Германия": ["Берлин", "Мюнхен"],
    "Италия": ["Рим", "Венеция", "Флоренция"]
}

parser.process_all(locations_to_parse)

In [2]:
import shutil

def recurcive_deletion(folder_name):
   shutil.rmtree(folder_name)

recurcive_deletion('./data')